# Trabalho 1 - EEL891

Etapas que serão realizadas a seguir:
- Implementar o pré-processamento
- Visualizar, analisar e selecionar os atributos
- Escolher os métodos que serão usados e compará-los
- Verificar o desempenho com validação cruzada
- Ajustas os hiperparâmetros para chegar ao melhor resultado possível
- Treinar o modelo com o conjunto de teste

### Importação das bibliotecas

In [1]:
import pandas as pd
import numpy  as np

from matplotlib                 import pyplot as plt

from sklearn.naive_bayes        import MultinomialNB, GaussianNB
from sklearn.model_selection    import cross_val_predict, train_test_split
from sklearn.metrics            import accuracy_score
from sklearn.neighbors          import KNeighborsClassifier
from sklearn.preprocessing      import LabelBinarizer, MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.linear_model       import LogisticRegression
from scipy.stats                import pearsonr

import sklearn

### Pré-processamento

Agora, o próximo passo é importar o conjunto de treinamento:

In [2]:
dados = pd.read_csv(r"C:\Users\andre\Documents\UFRJ\SÉTIMO PERÍODO\Introdução ao Aprendizado de Máquina\EEL891-Trabalho-1\conjunto_de_treinamento.csv")
dados = dados.iloc[:,:]

Embaralhamento dos dados e separá-los em atributos e alvo (convertendo para um array):

In [3]:
dados = dados.sample(frac=1)
x = dados.iloc[:,:-1].to_numpy()
y = dados.iloc[:,-1].to_numpy()

Descartando algumas informações que não são relevantes. O campo "id_solicitante" é um número de 1 a 20000, ou seja, apenas atrapalharia o modelo. Os campos "grau_instrucao" e "possui_telefone_celular" não foram extraídos corretamente, então segui a orientação do professor para descartá-los. E o campo "qtde_contas_bancarias_especiais" é exatamente igual ao campo "qtde_contas_bancarias", logo foi descartado seguindo a orientação do professor.

In [4]:
dados = dados.drop(['id_solicitante', 'grau_instrucao', 'possui_telefone_celular', 'qtde_contas_bancarias_especiais'],axis=1)

Identificação das variáveis categóricas olhando o dicionário de dados:

In [5]:
variaveisCategoricas = [x for x in dados.columns if dados[x].dtype=='object' or x == 'produto_solicitado' or x == 'tipo_endereco' or x =='estado_civil' or  x =='nacionalidade' or x=='tipo_residencia' or x=='possui_email' or x=='possui_cartao_visa' or x=='possui_cartao_mastercard' or x=='possui_cartao_diners' or x=='possui_cartao_amex' or x=='possui_cartao_cartoes' or x=='possui_carro' or x=='profissao' or x=='ocupacao' or x=='profissao_companheiro' or x=='local_onde_reside' or x=='local_onde_trabalha']
print(variaveisCategoricas)

['produto_solicitado', 'forma_envio_solicitacao', 'tipo_endereco', 'sexo', 'estado_civil', 'nacionalidade', 'estado_onde_nasceu', 'estado_onde_reside', 'possui_telefone_residencial', 'codigo_area_telefone_residencial', 'tipo_residencia', 'possui_email', 'possui_cartao_visa', 'possui_cartao_mastercard', 'possui_cartao_diners', 'possui_cartao_amex', 'possui_carro', 'vinculo_formal_com_empresa', 'estado_onde_trabalha', 'possui_telefone_trabalho', 'codigo_area_telefone_trabalho', 'profissao', 'ocupacao', 'profissao_companheiro', 'local_onde_reside', 'local_onde_trabalha']


Verificando as variáveis categóricas (quantidade de categorias e quais são):

In [6]:
for v in variaveisCategoricas:
    print("\n%15s: " %v, "%4d categorias" % len(dados[v].unique() ) )
    print(dados[v].unique(), "\n")


produto_solicitado:     3 categorias
[2 1 7] 


forma_envio_solicitacao:     3 categorias
['internet' 'presencial' 'correio'] 


  tipo_endereco:     2 categorias
[1 2] 


           sexo:     4 categorias
['F' 'M' ' ' 'N'] 


   estado_civil:     8 categorias
[2 6 1 4 0 5 3 7] 


  nacionalidade:     3 categorias
[1 0 2] 


estado_onde_nasceu:    28 categorias
['CE' 'RJ' 'MG' 'MT' 'RS' 'PE' 'PA' ' ' 'PB' 'SC' 'BA' 'SP' 'PR' 'RN' 'DF'
 'AP' 'GO' 'TO' 'ES' 'MA' 'AC' 'AL' 'AM' 'RO' 'SE' 'MS' 'PI' 'RR'] 


estado_onde_reside:    27 categorias
['CE' 'RJ' 'MG' 'MT' 'RS' 'PE' 'SP' 'GO' 'SC' 'BA' 'PA' 'PR' 'RN' 'DF'
 'AP' 'TO' 'ES' 'MA' 'PB' 'AL' 'AC' 'MS' 'SE' 'AM' 'RO' 'PI' 'RR'] 


possui_telefone_residencial:     2 categorias
['Y' 'N'] 


codigo_area_telefone_residencial:    81 categorias
['112' '20' ' ' '71' '58' '29' '5' '66' '50' '84' '12' '15' '97' '81'
 '117' '40' '107' '105' '54' '86' '123' '39' '32' '24' '62' '90' '120'
 '68' '85' '69' '25' '125' '14' '126' '103' '67' '27' '100' '

Correção de alguns valores (principalmente em branco). A ideia inicial é criar uma nova categoria para os valores em branco, e depois tentar excluí-los para ver qual é a resposta dos modelos:

In [7]:
for x in variaveisCategoricas:
    if x == 'sexo':
        dados[x].replace(' ', 'N', inplace=True)
    if x == 'estado_onde_nasceu':
        dados[x].replace(' ', 'NN', inplace=True)
    if x == 'codigo_area_telefone_residencial':
        dados[x].replace(' ', '000', inplace=True)
    if x == 'tipo_residencia':
        dados[x].replace(0, 6, inplace=True)
        dados[x].fillna(0, inplace=True)
    if x == 'estado_onde_trabalha':
        dados[x].replace(' ', 'NN', inplace=True)
    if x == 'codigo_area_telefone_trabalho':
        dados[x].replace(' ', '000', inplace=True)
    if x == 'profissao':
        dados[x].replace(0, 18, inplace=True)
        dados[x].fillna(0, inplace=True)
    if x == 'ocupacao':
        dados[x].replace(0, 6, inplace=True)
        dados[x].fillna(0, inplace=True)
    if x == 'profissao_companheiro':
        dados[x].replace(0, 18, inplace=True)
        dados[x].fillna(0, inplace=True)
    print("\n%15s: " %x, "%4d categorias" % len(dados[x].unique() ) )
    print(dados[x].unique(), "\n")


produto_solicitado:     3 categorias
[2 1 7] 


forma_envio_solicitacao:     3 categorias
['internet' 'presencial' 'correio'] 


  tipo_endereco:     2 categorias
[1 2] 


           sexo:     3 categorias
['F' 'M' 'N'] 


   estado_civil:     8 categorias
[2 6 1 4 0 5 3 7] 


  nacionalidade:     3 categorias
[1 0 2] 


estado_onde_nasceu:    28 categorias
['CE' 'RJ' 'MG' 'MT' 'RS' 'PE' 'PA' 'NN' 'PB' 'SC' 'BA' 'SP' 'PR' 'RN'
 'DF' 'AP' 'GO' 'TO' 'ES' 'MA' 'AC' 'AL' 'AM' 'RO' 'SE' 'MS' 'PI' 'RR'] 


estado_onde_reside:    27 categorias
['CE' 'RJ' 'MG' 'MT' 'RS' 'PE' 'SP' 'GO' 'SC' 'BA' 'PA' 'PR' 'RN' 'DF'
 'AP' 'TO' 'ES' 'MA' 'PB' 'AL' 'AC' 'MS' 'SE' 'AM' 'RO' 'PI' 'RR'] 


possui_telefone_residencial:     2 categorias
['Y' 'N'] 


codigo_area_telefone_residencial:    81 categorias
['112' '20' '000' '71' '58' '29' '5' '66' '50' '84' '12' '15' '97' '81'
 '117' '40' '107' '105' '54' '86' '123' '39' '32' '24' '62' '90' '120'
 '68' '85' '69' '25' '125' '14' '126' '103' '67' '27' '100' '3

In [8]:
for x in dados:
    if x == 'meses_na_residencia':
        dados[x].fillna(300, inplace=True)
    if x == 'grau_instrucao_companheiro':
        dados[x].fillna(6, inplace=True)
    print("\n%15s: " %x, "%4d categorias" % len(dados[x].unique() ) )
    print(dados[x].unique(), "\n")


produto_solicitado:     3 categorias
[2 1 7] 


 dia_vencimento:     6 categorias
[ 5 25 10 15  1 20] 


forma_envio_solicitacao:     3 categorias
['internet' 'presencial' 'correio'] 


  tipo_endereco:     2 categorias
[1 2] 


           sexo:     3 categorias
['F' 'M' 'N'] 


          idade:    84 categorias
[ 42  23  32  25  41  73  85  66  43  63  22  27  33  45  40  38  30  69
  55  29  24  34  35  19  58  47  21  51  36  70  26  53  65  37  54  31
  28  44  59  46  74  49  64  56  50  84  68  52  61  76  71  39  88  67
  48  60  57  20  80  62  78  18  77  79  91  72  82  75  17  81  89  86
  83  90  92 100  87  95  96  93  97  94   7 106] 


   estado_civil:     8 categorias
[2 6 1 4 0 5 3 7] 


qtde_dependentes:    15 categorias
[ 0  2  4  1  3  5  6  8  7 14  9 10 11 13 53] 


  nacionalidade:     3 categorias
[1 0 2] 


estado_onde_nasceu:    28 categorias
['CE' 'RJ' 'MG' 'MT' 'RS' 'PE' 'PA' 'NN' 'PB' 'SC' 'BA' 'SP' 'PR' 'RN'
 'DF' 'AP' 'GO' 'TO' 'ES' 'MA' 'AC' 'AL' 'AM' '

Binarizar e fazer o one hot encoding de alguns parâmetros:

In [9]:
dados = pd.get_dummies(dados, columns=['forma_envio_solicitacao',
                                       'sexo'
                                       #'estado_onde_nasceu',
                                       #'estado_onde_reside',
                                       #'estado_onde_trabalha'
                                      ])
#print(dados.T)

binarizador = LabelBinarizer()
for x in ['tipo_endereco',
          'possui_telefone_residencial',
          'vinculo_formal_com_empresa',
          'possui_telefone_trabalho']:
    dados[x] = binarizador.fit_transform(dados[x])
print(dados.T)

                                    8815   3084     5742    15369  14822  \
produto_solicitado                      2      1        1       1      1   
dia_vencimento                          5     25       10      25     25   
tipo_endereco                           0      0        0       0      0   
idade                                  42     23       32      25     41   
estado_civil                            2      2        2       2      2   
qtde_dependentes                        0      0        0       0      2   
nacionalidade                           1      1        1       1      1   
estado_onde_nasceu                     CE     RJ       MG      MT     RS   
estado_onde_reside                     CE     RJ       MG      MT     RS   
possui_telefone_residencial             1      1        0       1      1   
codigo_area_telefone_residencial      112     20      000      71     58   
tipo_residencia                       1.0    1.0      1.0     1.0    1.0   
meses_na_res

Verificar a correlação de pearson entre os parâmetros e o alvo (obs: os parâmetros não parecem muito promissores, mas veremos mais na hora de treinar o modelo):

In [10]:
colunas = dados.columns
for col in colunas:
    if type(dados[col][0]) != str:
        corr = pearsonr(dados[col], dados['inadimplente'])
        print(col, " = %.4f" % corr[0], " , %.10f" % corr[1])

produto_solicitado  = 0.0301  , 0.0000206384
dia_vencimento  = 0.0803  , 0.0000000000
tipo_endereco  = -0.0044  , 0.5332219466
idade  = -0.1207  , 0.0000000000
estado_civil  = -0.0313  , 0.0000097351
qtde_dependentes  = 0.0176  , 0.0126681749
nacionalidade  = 0.0005  , 0.9442714564
possui_telefone_residencial  = -0.0808  , 0.0000000000
tipo_residencia  = 0.0200  , 0.0046491853
meses_na_residencia  = -0.0047  , 0.5024577844
possui_email  = -0.0057  , 0.4168413604
renda_mensal_regular  = -0.0009  , 0.8958507584
renda_extra  = 0.0059  , 0.4079845863
possui_cartao_visa  = -0.0013  , 0.8559114105
possui_cartao_mastercard  = -0.0200  , 0.0047680534
possui_cartao_diners  = 0.0025  , 0.7234775228
possui_cartao_amex  = -0.0011  , 0.8757773534
possui_outros_cartoes  = -0.0032  , 0.6543764344
qtde_contas_bancarias  = 0.0141  , 0.0456463167
valor_patrimonio_pessoal  = 0.0012  , 0.8627919655
possui_carro  = 0.0128  , 0.0707060897
vinculo_formal_com_empresa  = 0.0062  , 0.3773057238
possui_telefone_

In [11]:
#atributos usados pelo classificador
atributosSelecionados = [
    'produto_solicitado',
    'dia_vencimento',
    'forma_envio_solicitacao_correio',
    'forma_envio_solicitacao_internet',
    'forma_envio_solicitacao_presencial',
    'tipo_endereco',
    'sexo_M',
    'sexo_F',
    'sexo_N',
    'idade',
    'estado_civil',
    'qtde_dependentes',
    'nacionalidade',
    #'estado_onde_nasceu',
    #'estado_onde_reside',
    'possui_telefone_residencial',
    #'codigo_area_telefone_residencial',
    'tipo_residencia',
    'meses_na_residencia',
    'possui_email',
    'renda_mensal_regular',
    'renda_extra',
    'possui_cartao_visa',
    'possui_cartao_mastercard',
    'possui_cartao_diners',
    'possui_cartao_amex',
    'possui_outros_cartoes',
    'qtde_contas_bancarias',
    'valor_patrimonio_pessoal',
    'possui_carro',
    'vinculo_formal_com_empresa',
    #'estado_onde_trabalha',
    'possui_telefone_trabalho',
    #'codigo_area_telefone_trabalho',
    'meses_no_trabalho',
    'profissao',
    'ocupacao',
    'profissao_companheiro',
    'grau_instrucao_companheiro',
    #'local_onde_reside',
    #'local_onde_trabalha',
    'inadimplente'
]

alvo = 'inadimplente'

#embaralhar o conjunto de dados
dadosEmbaralhados = dados[atributosSelecionados].sample(frac=1, random_state=1)

#criar arrays x e y separando os parâmetros do alvo
x = dadosEmbaralhados.loc[:,dadosEmbaralhados.columns!=alvo].values
y = dadosEmbaralhados.loc[:,dadosEmbaralhados.columns==alvo].values

#separar em conjunto de treino e de teste
amostrasTreinamento = 15000

xTreino = x[:amostrasTreinamento,:]
yTreino = y[:amostrasTreinamento].ravel()

xTeste = x[amostrasTreinamento:,:]
yTeste = y[amostrasTreinamento:].ravel()

#ajustar a escala
scaler = MinMaxScaler()
scaler.fit(xTreino)

xTreino = scaler.transform(xTreino)
xTeste = scaler.transform(xTeste)

#verificar a variacao da acuracia para regressao logistica

#verificar a variacao da acuracia com o numero de vizinhos
print("\n  K  TREINO  TESTE")
print(" --  ------  ------")

for k in range(1,50):
    #treinar classificador KNN
    classificador = KNeighborsClassifier(
        n_neighbors=k,
        weights = 'distance',
        p=1
    )
    classificador = classificador.fit(xTreino, yTreino)
    
    #obter as respostas do classificador
    yRespostaTreino = classificador.predict(xTreino)
    yRespostaTeste = classificador.predict(xTeste)
    
    #desempenho na amostra de treino e teste
    acuraciaTreino = sum(yRespostaTreino==yTreino)/len(yTreino)
    acuraciaTeste = sum(yRespostaTeste==yTeste)/len(yTeste)
    
    print(
        "%3d" % k,
        "%6.1f" % (100*acuraciaTreino),
        "%6.1f" % (100*acuraciaTeste)
    )
    



  K  TREINO  TESTE
 --  ------  ------
  1  100.0   54.4
  2  100.0   54.4
  3  100.0   53.9
  4  100.0   54.4
  5  100.0   55.0
  6  100.0   54.9
  7  100.0   54.5
  8  100.0   55.3
  9  100.0   55.3
 10  100.0   55.9
 11  100.0   56.0
 12  100.0   55.8
 13  100.0   56.1
 14  100.0   56.1
 15  100.0   56.0
 16  100.0   56.3
 17  100.0   56.3
 18  100.0   56.3
 19  100.0   56.5
 20  100.0   56.9
 21  100.0   56.6
 22  100.0   57.0
 23  100.0   56.7
 24  100.0   56.6
 25  100.0   56.7
 26  100.0   56.6
 27  100.0   56.8
 28  100.0   56.9
 29  100.0   56.7
 30  100.0   56.9
 31  100.0   57.2
 32  100.0   57.0
 33  100.0   57.0
 34  100.0   57.3
 35  100.0   57.0
 36  100.0   57.1
 37  100.0   57.2
 38  100.0   57.3
 39  100.0   57.1
 40  100.0   57.5
 41  100.0   57.7
 42  100.0   57.5
 43  100.0   57.4
 44  100.0   57.6
 45  100.0   57.5
 46  100.0   57.5
 47  100.0   57.2
 48  100.0   57.1
 49  100.0   57.0


### Treinamento do modelo

Classificador Bayesiano com Distribuição Multinomial:

In [12]:
classificador = MultinomialNB(alpha=1.0)
y_pred = cross_val_predict(classificador, x, y, cv=5)

print("Acurácia = %6.4f" % accuracy_score(y,y_pred))

C:\Users\andre\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\andre\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\andre\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\andre\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

Acurácia = 0.4966


Classificador Bayesiano com Distribuição Multinomial:

In [13]:
classificador = GaussianNB()
y_pred = cross_val_predict(classificador, x, y, cv=5)

print("Acurácia = %6.4f" % accuracy_score(y,y_pred))

C:\Users\andre\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\andre\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\andre\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\andre\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

Acurácia = 0.4973


C:\Users\andre\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Classificador KNN

In [ ]:
print("\n  K  TREINO  TESTE")
print(" --  ------  ------")

for k in range(1,50):
    classificador = KNeighborsClassifier(
        n_neighbors=k,
        weights = 'uniform',
        p=1
    )
    classificador = classificador.fit(xTreino, yTreino)
    
    yRespostaTreino = classificador.predict(xTreino)
    yRespostaTeste = classificador.predict(xTeste)
    
    acuraciaTreino = sum(yRespostaTreino==yTreino)/len(yTreino)
    acuraciaTeste = sum(yRespostaTeste==yTeste)/len(yTeste)
    
    print(
        "%3d" % k,
        "%6.1f" % (100*acuraciaTreino),
        "%6.1f" % (100*acuraciaTeste)
    )
    



  K  TREINO  TESTE
 --  ------  ------
  1  100.0   54.4


Classificador de Regressão Logística

In [ ]:
print("\n         C  TREINO  TESTE")
print(" ----------  ------  ------")

for k in range(-6,7):
    c = pow(10,k)
    classificador = LogisticRegression(
        penalty = 'l2',
        C = c
    )
    print(yTreino)
    classificador = classificador.fit(xTreino, yTreino)
    
    yRespostaTreino = classificador.predict(xTreino)
    yRespostaTeste = classificador.predict(xTeste)
    
    acuraciaTreino = sum(yRespostaTreino==yTreino)/len(yTreino)
    acuraciaTeste = sum(yRespostaTeste==yTeste)/len(yTeste)
    
    print(
        "%14.6f" % c,
        "%6.1f" % (100*acuraciaTreino),
        "%6.1f" % (100*acuraciaTeste)
    )